# edges - Biodiversity, IBIF

https://www.nature.com/articles/s41597-025-05946-1#Sec14

https://doi.org/10.5281/zenodo.16234138

In [1]:
import pandas as pd
import country_converter as coco
import json
import warnings
warnings.filterwarnings('ignore')

## 1. Load data

In [2]:
cfs_country = pd.read_excel("data/IBIF_v2.xlsx", sheet_name = 'BIFs per pressure')
cfs_country.head()

,ISO3,CO2_plants,CO2_vert,CO2_overall,NH3_plants,NH3_overall,NOx_plants,NOx_overall,LU_Cr_plants,LU_Cr_vert,...,LU_Pl_vert,LU_Pl_overall,LU_Ur_plants,LU_Ur_vert,LU_Ur_overall,LU_Mi_plants,LU_Mi_vert,LU_Mi_overall,R_vert,R_overall
0,AFG,4.720000e-07,3.090000e-07,3.900000e-07,0.000039,0.000020,0.000013,0.000006,0.743,0.568,...,NaN,NaN,0.688,0.757,0.722,0.915,3.77,2.34,0.783000,0.392000
1,ALB,4.720000e-07,3.090000e-07,3.900000e-07,0.000065,0.000032,0.000025,0.000012,0.743,0.500,...,NaN,NaN,0.688,0.746,0.717,0.915,2.31,1.61,0.391000,0.195000
2,DZA,4.720000e-07,3.090000e-07,3.900000e-07,0.000094,0.000047,0.000042,0.000021,0.743,0.533,...,0.361,0.466,0.688,0.725,0.706,0.915,2.94,1.93,0.984000,0.492000
3,ASM,4.720000e-07,3.090000e-07,3.900000e-07,0.000049,0.000025,0.000014,0.000007,0.743,0.604,...,NaN,NaN,0.688,0.783,0.736,NaN,NaN,NaN,0.000158,0.000079
4,AND,4.720000e-07,3.090000e-07,3.900000e-07,0.000039,0.000020,0.000018,0.000009,0.743,0.562,...,NaN,NaN,0.688,0.758,0.723,NaN,NaN,NaN,0.855000,0.428000


In [3]:
cfs_country['ISO3'] = cfs_country['ISO3'].replace({
    'ANT': 'BES',        # Caribbean Netherlands (Bonaire, Sint Eustatius, Saba)
})

## 2. Matching parameters to environmental flows

In [4]:
cfs_country.columns

Index(['ISO3', 'CO2_plants', 'CO2_vert', 'CO2_overall', 'NH3_plants',
       'NH3_overall', 'NOx_plants', 'NOx_overall', 'LU_Cr_plants',
       'LU_Cr_vert', 'LU_Cr_overall', 'LU_Pa_plants', 'LU_Pa_vert',
       'LU_Pa_overall', 'LU_Pl_plants', 'LU_Pl_vert', 'LU_Pl_overall',
       'LU_Ur_plants', 'LU_Ur_vert', 'LU_Ur_overall', 'LU_Mi_plants',
       'LU_Mi_vert', 'LU_Mi_overall', 'R_vert', 'R_overall'],
      dtype='object')

In [5]:
# import bw2data as bd
# import bw2io as bi
# bd.projects.set_current("bw25_ei311")
# bf = bd.Database('biosphere')
# sorted([m["name"] for m in bf if "Nitrogen oxides" in m["name"]])

In [6]:
mapping_dict = {
    'Carbon dioxide, fossil': 'CO2_',
    'Carbon dioxide, from soil or biomass stock': 'CO2_',
    'Carbon dioxide, in air': 'CO2_',
    'Carbon dioxide, non-fossil': 'CO2_',
    'Carbon dioxide, to soil or biomass stock': 'CO2_',
    'Ammonia': 'NH3_',
    'Nitrogen oxides': 'NOx_',
    'Nitrogen dioxide': 'NOx_',    
    # 'LU_Cr_', # cropland
    # 'LU_Pa_', # pasture
    # 'LU_Pl_', # plantation forest
    # 'LU_Ur_', # urban land
    # 'LU_Mi_', # mines
    'Occupation, annual crop': 'LU_Cr_',
    'Occupation, annual crop, flooded crop': 'LU_Cr_',
    'Occupation, annual crop, greenhouse': 'LU_Ur_',
    'Occupation, annual crop, irrigated': 'LU_Cr_',
    'Occupation, annual crop, irrigated, extensive': 'LU_Cr_',
    'Occupation, annual crop, irrigated, intensive': 'LU_Cr_',
    'Occupation, annual crop, non-irrigated': 'LU_Cr_',
    'Occupation, annual crop, non-irrigated, extensive': 'LU_Cr_',
    'Occupation, annual crop, non-irrigated, intensive': 'LU_Cr_',
    'Occupation, arable land, unspecified use': 'LU_Cr_',
    # 'Occupation, arable, conservation tillage (obsolete)': '',
    # 'Occupation, arable, conventional tillage (obsolete)': '',
    # 'Occupation, arable, reduced tillage (obsolete)': '',
    # 'Occupation, bare area (non-use)': '',
    'Occupation, construction site': 'LU_Ur_',
    'Occupation, cropland fallow (non-use)': 'LU_Cr_',
    'Occupation, dump site': 'LU_Ur_',
    'Occupation, field margin/hedgerow': 'LU_Cr_',
    'Occupation, forest, extensive': 'LU_Pa_',
    'Occupation, forest, intensive': 'LU_Pl_',
    'Occupation, forest, primary (non-use)': 'LU_Pl_',
    'Occupation, forest, secondary (non-use)': 'LU_Pa_',
    'Occupation, forest, unspecified': 'LU_Pl_',
    'Occupation, grassland, natural (non-use)': 'LU_Pa_',
    'Occupation, grassland, natural, for livestock grazing': 'LU_Pa_',
    'Occupation, heterogeneous, agricultural': 'LU_Cr_',
    'Occupation, industrial area': 'LU_Ur_',
    # 'Occupation, inland waterbody, unspecified': '',
    # 'Occupation, lake, artificial': '',
    # 'Occupation, lake, natural (non-use)': '',
    'Occupation, mineral extraction site': 'LU_Mi_',
    'Occupation, pasture, man made': 'LU_Pa_',
    'Occupation, pasture, man made, extensive': 'LU_Pa_',
    'Occupation, pasture, man made, intensive': 'LU_Pa_',
    'Occupation, permanent crop': 'LU_Cr_',
    'Occupation, permanent crop, irrigated': 'LU_Cr_',
    'Occupation, permanent crop, irrigated, extensive': 'LU_Cr_',
    'Occupation, permanent crop, irrigated, intensive': 'LU_Cr_',
    'Occupation, permanent crop, non-irrigated': 'LU_Cr_',
    'Occupation, permanent crop, non-irrigated, extensive': 'LU_Cr_',
    'Occupation, permanent crop, non-irrigated, intensive': 'LU_Cr_',
    # 'Occupation, river, artificial': '',
    # 'Occupation, river, natural (non-use)': '',
    # 'Occupation, seabed, drilling and mining': '',
    # 'Occupation, seabed, infrastructure': '',
    # 'Occupation, seabed, natural (non-use)': '',
    # 'Occupation, seabed, unspecified': '',
    'Occupation, shrub land, sclerophyllous': 'LU_Pl_',
    # 'Occupation, snow and ice (non-use)': '',
    'Occupation, traffic area, rail network': 'LU_Ur_',
    'Occupation, traffic area, rail/road embankment': 'LU_Ur_',
    'Occupation, traffic area, road network': 'LU_Ur_',
    'Occupation, unspecified': 'LU_Ur_',
    'Occupation, unspecified, natural (non-use)': 'LU_Pa_',
    'Occupation, urban, continuously built': 'LU_Ur_',
    'Occupation, urban, discontinuously built': 'LU_Ur_',
    'Occupation, urban, green area': 'LU_Ur_',
    'Occupation, urban/industrial fallow (non-use)': 'LU_Ur_',
    # 'Occupation, wetland, coastal (non-use)': '',
    # 'Occupation, wetland, inland (non-use)': '',
}

In [7]:
emission_flows = {
    'Carbon dioxide, fossil': ('CO2_', 1.0),
    'Carbon dioxide, from soil or biomass stock': ('CO2_', 1.0),
    'Carbon dioxide, in air': ('CO2_', 1.0),
    'Carbon dioxide, non-fossil': ('CO2_', 1.0),
    'Carbon dioxide, to soil or biomass stock': ('CO2_', 1.0),
    'Methane, fossil': ('CO2_', 29.8),
    'Methane, from soil or biomass stock': ('CO2_', 27.0),
    'Methane, non-fossil': ('CO2_', 27.0),
    'Dinitrogen monoxide': ('CO2_', 273.0),
    'Ammonia': ('NH3_', 1.0),
    'Nitrogen oxides': ('NOx_', 1.0),
    'Nitrogen dioxide': ('NOx_', 1.0),
}
emission_flows

{'Carbon dioxide, fossil': ('CO2_', 1.0),
 'Carbon dioxide, from soil or biomass stock': ('CO2_', 1.0),
 'Carbon dioxide, in air': ('CO2_', 1.0),
 'Carbon dioxide, non-fossil': ('CO2_', 1.0),
 'Carbon dioxide, to soil or biomass stock': ('CO2_', 1.0),
 'Methane, fossil': ('CO2_', 29.8),
 'Methane, from soil or biomass stock': ('CO2_', 27.0),
 'Methane, non-fossil': ('CO2_', 27.0),
 'Dinitrogen monoxide': ('CO2_', 273.0),
 'Ammonia': ('NH3_', 1.0),
 'Nitrogen oxides': ('NOx_', 1.0),
 'Nitrogen dioxide': ('NOx_', 1.0)}

In [8]:
occupation_flows = {k: v for k, v in mapping_dict.items() if k not in [name for name in emission_flows.keys() if emission_flows[name][1] == 1.0]}
occupation_flows

{'Occupation, annual crop': 'LU_Cr_',
 'Occupation, annual crop, flooded crop': 'LU_Cr_',
 'Occupation, annual crop, greenhouse': 'LU_Ur_',
 'Occupation, annual crop, irrigated': 'LU_Cr_',
 'Occupation, annual crop, irrigated, extensive': 'LU_Cr_',
 'Occupation, annual crop, irrigated, intensive': 'LU_Cr_',
 'Occupation, annual crop, non-irrigated': 'LU_Cr_',
 'Occupation, annual crop, non-irrigated, extensive': 'LU_Cr_',
 'Occupation, annual crop, non-irrigated, intensive': 'LU_Cr_',
 'Occupation, arable land, unspecified use': 'LU_Cr_',
 'Occupation, construction site': 'LU_Ur_',
 'Occupation, cropland fallow (non-use)': 'LU_Cr_',
 'Occupation, dump site': 'LU_Ur_',
 'Occupation, field margin/hedgerow': 'LU_Cr_',
 'Occupation, forest, extensive': 'LU_Pa_',
 'Occupation, forest, intensive': 'LU_Pl_',
 'Occupation, forest, primary (non-use)': 'LU_Pl_',
 'Occupation, forest, secondary (non-use)': 'LU_Pa_',
 'Occupation, forest, unspecified': 'LU_Pl_',
 'Occupation, grassland, natural (n

## 3. Generate .json files

In [9]:
cc = coco.CountryConverter()

In [10]:
import wbgapi as wb

def get_gdp_data():
    """Fetch GDP data from World Bank for all countries, convert to ISO2"""
    
    # Manual GDP data for countries missing from World Bank (in current USD, recent estimates)
    MANUAL_GDP = {
        'TW': 751000000000,      # Taiwan (2023 est.)
        'VE': 269000000000,      # Venezuela (2023 est.)
        'YE': 21000000000,       # Yemen (2023 est.)
        'ER': 2000000000,        # Eritrea (2023 est.)
        'KP': 28000000000,       # North Korea (2023 est.)
        'SS': 11000000000,       # South Sudan (2023 est.)
    }
    
    # Small territories/regions where missing GDP is expected (no warning needed)
    KNOWN_MISSING_GDP = {
        'IO', 'VG', 'CX', 'CC', 'YT', 'CK', 'FK', 'GF', 'TF', 'GI', 'GP', 
        'MQ', 'MS', 'BQ', 'NU', 'NF', 'PN', 'RE', 'BL', 'SH', 'AI', 'PM', 
        'EH', 'SJ', 'WF', 'GG',
    }
    
    try:
        # Get GDP (current US$) - indicator NY.GDP.MKTP.CD
        gdp_data = wb.data.DataFrame('NY.GDP.MKTP.CD', time=range(2020, 2024), skipBlanks=True, labels=True)
        
        # Get the most recent year with data for each country
        gdp_dict_iso3 = {}
        for country in gdp_data.index:
            # Get most recent non-null value
            values = gdp_data.loc[country].dropna()
            if len(values) > 0:
                gdp_dict_iso3[country] = float(values.iloc[-1])
        
        # Convert ISO3 to ISO2, filtering out aggregates
        gdp_dict_iso2 = {}
        skipped_aggregates = 0
        
        for iso3, gdp in gdp_dict_iso3.items():
            try:
                iso2 = cc.convert(names=iso3, to='ISO2')
                if iso2 != 'not found':
                    gdp_dict_iso2[iso2] = gdp
                else:
                    skipped_aggregates += 1
            except:
                skipped_aggregates += 1
        
        # Add manual GDP data
        gdp_dict_iso2.update(MANUAL_GDP)
        
        print(f"Skipped {skipped_aggregates} regional/aggregate codes (not actual countries)")
        print(f"Added manual GDP data for {len(MANUAL_GDP)} countries")
        
        # Return both the GDP dict and the set of known missing countries
        return gdp_dict_iso2, KNOWN_MISSING_GDP
        
    except Exception as e:
        print(f"Error fetching GDP data: {e}")
        return {}, set()

# Get GDP data once
print("Fetching GDP data from World Bank...")
gdp_by_iso2, KNOWN_MISSING_GDP = get_gdp_data()
print(f"GDP data loaded for {len(gdp_by_iso2)} countries")

Fetching GDP data from World Bank...


CHI not found in ISO3
WLD not found in ISO3
UMC not found in ISO3
TSS not found in ISO3
SSA not found in ISO3
SSF not found in ISO3
TSA not found in ISO3
SAS not found in ISO3
SST not found in ISO3
PRE not found in ISO3
PST not found in ISO3
PSS not found in ISO3
OSS not found in ISO3
OED not found in ISO3
NAC not found in ISO3
MIC not found in ISO3
TMN not found in ISO3
MNA not found in ISO3
MEA not found in ISO3
LMC not found in ISO3
LIC not found in ISO3
LMY not found in ISO3
LDC not found in ISO3
TLA not found in ISO3
LAC not found in ISO3
LCN not found in ISO3
LTE not found in ISO3
IDA not found in ISO3
IDX not found in ISO3
IDB not found in ISO3
IBT not found in ISO3
IBD not found in ISO3
HIC not found in ISO3
HPC not found in ISO3
FCS not found in ISO3
EUU not found in ISO3
TEC not found in ISO3
ECA not found in ISO3
ECS not found in ISO3
EMU not found in ISO3
TEA not found in ISO3
EAP not found in ISO3
EAS not found in ISO3
EAR not found in ISO3
CEB not found in ISO3
CSS not fo

Skipped 49 regional/aggregate codes (not actual countries)
Added manual GDP data for 6 countries
GDP data loaded for 215 countries


In [11]:
gdp_by_iso2

{'ZW': 35231369343.4002,
 'ZM': 27577956471.244,
 'PS': 17847900000.0,
 'VI': 4672000000.0,
 'VN': 433857681378.291,
 'VU': 1126313359.21923,
 'UZ': 102641879248.764,
 'UY': 77991666837.7382,
 'US': 27720709000000.0,
 'GB': 3369861888972.88,
 'AE': 514130432648.06,
 'UA': 181221517868.744,
 'UG': 48768955859.2834,
 'TV': 62280311.5852172,
 'TC': 1638653000.0,
 'TM': 60628857142.8571,
 'TR': 1118252958513.72,
 'TN': 48196281783.9479,
 'TT': 25489373405.0619,
 'TO': 508735107.033734,
 'TG': 9171261837.57103,
 'TL': 2079916909.49485,
 'TH': 515906283940.933,
 'TZ': 79062403836.8432,
 'TJ': 12244169292.7998,
 'SY': 19993439949.5435,
 'CH': 894424821645.205,
 'SE': 585489968189.064,
 'SR': 3455146280.83975,
 'SD': 39898289820.7951,
 'VC': 1072237037.03704,
 'MF': 649206156.194207,
 'LC': 2430166666.66667,
 'KN': 1056751851.85185,
 'LK': 83716142582.1945,
 'ES': 1620090734956.89,
 'ZA': 380699271816.395,
 'SO': 10968517089.9982,
 'SB': 1660896530.71894,
 'SI': 69148468417.3208,
 'SK': 133896

In [12]:
def generate_emission_json(species_group, pressure_type=None):
    """
    Generate JSON for emission flows (CO2, NH3, NOx)
    
    Parameters:
    -----------
    species_group : str
        One of 'plants', 'vert', or 'overall'
    pressure_type : str, optional
        One of 'CO2', 'NH3', 'NOx', or None for all emissions
    """

    species_group_names = {
        'plants': 'plants',
        'vert': 'vertebrates',
        'overall': 'overall'
    }
    
    exchanges = []
    
    # Determine which emission flows to process
    if pressure_type == 'CO2':
        flows_to_process = {k: v for k, v in emission_flows.items() if v[0] == 'CO2_'}
    elif pressure_type == 'NH3':
        flows_to_process = {k: v for k, v in emission_flows.items() if v[0] == 'NH3_'}
    elif pressure_type == 'NOx':
        flows_to_process = {k: v for k, v in emission_flows.items() if v[0] == 'NOx_'}
    else:  # All emissions
        flows_to_process = emission_flows
    
    for flow_name, (prefix, conversion_factor) in flows_to_process.items():
        # Get column name for this species group
        col_name = f"{prefix}{species_group}"
        
        if col_name not in cfs_country.columns:
            continue
        
        # Get all countries
        for _, row in cfs_country.iterrows():
            country_iso3 = row['ISO3']
            cf_value = row[col_name]
            
            # Skip if CF is NaN or zero
            if pd.isna(cf_value) or cf_value == 0:
                continue
            
            # Apply conversion factor for non-CO2 greenhouse gases
            cf_value = cf_value * conversion_factor
            
            # Convert ISO3 to ISO2
            try:
                country_iso2 = cc.convert(names=country_iso3, to='ISO2')
                if country_iso2 == 'not found':
                    # print(f"Warning: Could not convert {country_iso3} to ISO2")
                    continue
            except:
                # print(f"Warning: Could not convert {country_iso3} to ISO2")
                continue
            
            if country_iso2 in gdp_by_iso2:
                gdp = gdp_by_iso2[country_iso2]
            elif country_iso2 in KNOWN_MISSING_GDP:
                gdp = 0  # Known small territory, no warning
            else:
                print(f"WARNING: No GDP data for {country_iso2} ({country_iso3}), setting weight to 0")
                gdp = 0
            
            exchange = {
                "supplier": {
                    "name": flow_name,
                    # "type": "emission",
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": country_iso2,
                    "matrix": "technosphere"
                },
                "value": float(cf_value),
                "weight": float(gdp)
            }
            
            exchanges.append(exchange)
    
    # Set appropriate unit and description
    unit = 'MSA-loss.km2.yr'
    if pressure_type == 'CO2':
        pressure_desc = "greenhouse gas emissions (CO2, CH4, N2O as CO2-eq)"
    elif pressure_type == 'NH3':
        pressure_desc = "NH3 emissions"
    elif pressure_type == 'NOx':
        pressure_desc = "NOx emissions"
    else:
        pressure_desc = "atmospheric emissions (GHG as CO2-eq, NH3, NOx)"
    
    output = {
        "name": f"IBIF v2 - {pressure_desc} - biodiversity intactness - {species_group_names[species_group]}",
        "unit": unit,
        "version": "2.0",
        "description": f"Intactness-based Biodiversity Impact Factors (IBIF) for {pressure_desc} on terrestrial biodiversity intactness, ({species_group_names[species_group]}). Based on GLOBIO 4 model and Mean Species Abundance (MSA) metric. From Schipper et al. 2025: https://doi.org/10.1038/s41597-025-05946-1. Country-level entries include a GDP-based weight used to compute weighted averages when aggregating to composite or unspecified regions.",
        "species_group": species_group_names[species_group],
        "pressure_type": pressure_type if pressure_type else "emissions",
        "strategies": [
            "map_exchanges",
            "map_aggregate_locations",
            "map_dynamic_locations",
            "map_contained_locations",
            "map_remaining_locations_to_global"
        ],
        "exchanges": exchanges
    }
    
    return output

In [13]:
def generate_occupation_json(species_group):
    """
    Generate JSON for land occupation flows (all LU types)
    
    Parameters:
    -----------
    species_group : str
        One of 'plants', 'vert', or 'overall'
    """

    species_group_names = {
        'plants': 'plants',
        'vert': 'vertebrates',
        'overall': 'overall'
    }
    
    exchanges = []
    
    for flow_name, lu_prefix in occupation_flows.items():
        # Get column name for this species group and land use type
        col_name = f"{lu_prefix}{species_group}"
        
        if col_name not in cfs_country.columns:
            continue
        
        # Get all countries
        for _, row in cfs_country.iterrows():
            country_iso3 = row['ISO3']
            cf_value = row[col_name]
            
            # Skip if CF is NaN or zero
            if pd.isna(cf_value) or cf_value == 0:
                continue
            
            # Convert from MSA·km²/km² to MSA·km²/m²
            # Since the unit is km²/km², we need to convert the denominator from km² to m²
            cf_value_m2 = cf_value / 1_000_000
            
            # Convert ISO3 to ISO2
            try:
                country_iso2 = cc.convert(names=country_iso3, to='ISO2')
                if country_iso2 == 'not found':
                    # print(f"Warning: Could not convert {country_iso3} to ISO2")
                    continue
            except:
                # print(f"Warning: Could not convert {country_iso3} to ISO2")
                continue
            
            
            if country_iso2 in gdp_by_iso2:
                gdp = gdp_by_iso2[country_iso2]
            elif country_iso2 in KNOWN_MISSING_GDP:
                gdp = 0  # Known small territory, no warning
            else:
                print(f"WARNING: No GDP data for {country_iso2} ({country_iso3}), setting weight to 0")
                gdp = 0
                
            exchange = {
                "supplier": {
                    "name": flow_name,
                    "categories": ["natural resource", "land"],
                    # "type": "natural resource",
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": country_iso2,
                    "matrix": "technosphere"
                },
                "value": float(cf_value_m2),
                "weight": float(gdp)
            }
            
            exchanges.append(exchange)
    
    # Set unit and description
    unit = 'MSA-loss.km2.yr'
    pressure_desc = "land occupation"
    
    output = {
        "name": f"IBIF v2 - {pressure_desc} - biodiversity intactness - {species_group_names[species_group]}",
        "unit": unit,
        "version": "2.0",
        "description": f"Intactness-based Biodiversity Impact Factors (IBIF) for {pressure_desc} impacts on terrestrial biodiversity intactness, ({species_group_names[species_group]}). Includes all land use types: cropland, pasture, plantation forest, urban land, and mines. Based on GLOBIO 4 model and Mean Species Abundance (MSA) metric. Original units MSA·km2/km2 converted to MSA·km2/m2 to match ecoinvent land occupation flows (m2·yr). From Schipper et al. 2025: https://doi.org/10.1038/s41597-025-05946-1. Country-level entries include a GDP-based weight used to compute weighted averages when aggregating to composite or unspecified regions.",
        "species_group": species_group_names[species_group],
        "pressure_type": "land_occupation",
        "strategies": [
            "map_exchanges",
            "map_aggregate_locations",
            "map_dynamic_locations",
            "map_contained_locations",
            "map_remaining_locations_to_global"
        ],
        "exchanges": exchanges
    }
    
    return output

In [14]:
def generate_all_pressures_json(species_group):
    """
    Generate JSON combining all pressure types (emissions + land occupation)
    
    Parameters:
    -----------
    species_group : str
        One of 'plants', 'vert', or 'overall'
    """
    
    exchanges = []

    species_group_names = {
        'plants': 'plants',
        'vert': 'vertebrates',
        'overall': 'overall'
    }
    
    # Add all emission exchanges
    for flow_name, (prefix, conversion_factor) in emission_flows.items():
        col_name = f"{prefix}{species_group}"
        
        if col_name not in cfs_country.columns:
            continue
        
        for _, row in cfs_country.iterrows():
            country_iso3 = row['ISO3']
            cf_value = row[col_name]
            
            if pd.isna(cf_value) or cf_value == 0:
                continue
            
            # Apply conversion factor for non-CO2 greenhouse gases
            cf_value = cf_value * conversion_factor
            
            try:
                country_iso2 = cc.convert(names=country_iso3, to='ISO2')
                if country_iso2 == 'not found':
                    continue
            except:
                continue

            if country_iso2 in gdp_by_iso2:
                gdp = gdp_by_iso2[country_iso2]
            elif country_iso2 in KNOWN_MISSING_GDP:
                gdp = 0  # Known small territory, no warning
            else:
                print(f"WARNING: No GDP data for {country_iso2} ({country_iso3}), setting weight to 0")
                gdp = 0
            
            exchange = {
                "supplier": {
                    "name": flow_name,
                    # "type": "emission",
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": country_iso2,
                    "matrix": "technosphere"
                },
                "value": float(cf_value),
                "weight": float(gdp)
            }
            
            exchanges.append(exchange)
    
    # Add all occupation exchanges
    for flow_name, lu_prefix in occupation_flows.items():
        col_name = f"{lu_prefix}{species_group}"
        
        if col_name not in cfs_country.columns:
            continue
        
        for _, row in cfs_country.iterrows():
            country_iso3 = row['ISO3']
            cf_value = row[col_name]
            
            if pd.isna(cf_value) or cf_value == 0:
                continue
            
            # Convert from MSA·km²/km² to MSA·km²/m²
            cf_value_m2 = cf_value / 1_000_000
            
            try:
                country_iso2 = cc.convert(names=country_iso3, to='ISO2')
                if country_iso2 == 'not found':
                    continue
            except:
                continue
            
            if country_iso2 in gdp_by_iso2:
                gdp = gdp_by_iso2[country_iso2]
            elif country_iso2 in KNOWN_MISSING_GDP:
                gdp = 0  # Known small territory, no warning
            else:
                print(f"WARNING: No GDP data for {country_iso2} ({country_iso3}), setting weight to 0")
                gdp = 0
            
            exchange = {
                "supplier": {
                    "name": flow_name,
                    # "type": "natural resource",
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": country_iso2,
                    "matrix": "technosphere"
                },
                "value": float(cf_value_m2),
                "weight": float(gdp)
            }
            
            exchanges.append(exchange)
    
    unit = 'MSA-loss.km2.yr'
    output = {
        "name": f"IBIF v2 - all pressures - biodiversity intactness - {species_group_names[species_group]}",
        "unit": unit,
        "version": "2.0",
        "description": f"Intactness-based Biodiversity Impact Factors (IBIF) for all environmental pressures (GHG as CO2-eq, NH3, NOx emissions and land occupation) impacts on terrestrial biodiversity intactness, ({species_group_names[species_group]}). Land occupation converted from km2 to m2 to match ecoinvent units. Based on GLOBIO 4 model and Mean Species Abundance (MSA) metric. From Schipper et al. 2025: https://doi.org/10.1038/s41597-025-05946-1. Country-level entries include a GDP-based weight used to compute weighted averages when aggregating to composite or unspecified regions.",
        "species_group": species_group_names[species_group],
        "pressure_type": "all",
        "strategies": [
            "map_exchanges",
            "map_aggregate_locations",
            "map_dynamic_locations",
            "map_contained_locations",
            "map_remaining_locations_to_global"
        ],
        "exchanges": exchanges
    }
    
    return output

In [15]:
species_groups = ['plants', 'vert', 'overall']

species_group_names = {
    'plants': 'plants',
    'vert': 'vertebrates',
    'overall': 'overall'
}

# Generate files for each species group
for species_group in species_groups:
    print(f"\nProcessing {species_group_names[species_group]}...")

    full_name = species_group_names[species_group]
    
    # 1. Individual pressure emissions (CO2, NH3, NOx)
    for pressure in ['CO2', 'NH3', 'NOx']:
        json_data = generate_emission_json(species_group, pressure)
        if len(json_data['exchanges']) > 0:
            filename = f"IBIF_biodiversity_{pressure}_{full_name}.json"
            with open(filename, 'w') as f:
                json.dump(json_data, f, indent=2)
            print(f"  Created: {filename} ({len(json_data['exchanges'])} exchanges)")
        else:
            print(f"  Skipped: IBIF_biodiversity_{pressure}_{full_name}.json (0 exchanges)")
    
    # 2. Land occupation (all LU types combined)
    json_data = generate_occupation_json(species_group)
    if len(json_data['exchanges']) > 0:
        filename = f"IBIF_biodiversity_LU_{full_name}.json"
        with open(filename, 'w') as f:
            json.dump(json_data, f, indent=2)
        print(f"  Created: {filename} ({len(json_data['exchanges'])} exchanges)")
    else:
        print(f"  Skipped: IBIF_LU_{full_name}.json (0 exchanges)")
    
    # 3. All pressures combined
    json_data = generate_all_pressures_json(species_group)
    if len(json_data['exchanges']) > 0:
        filename = f"IBIF_biodiversity_all pressures_{full_name}.json"
        with open(filename, 'w') as f:
            json.dump(json_data, f, indent=2)
        print(f"  Created: {filename} ({len(json_data['exchanges'])} exchanges)")
    else:
        print(f"  Skipped: IBIF_biodiversity_all pressures_{full_name}.json (0 exchanges)")

print("\n" + "="*60)
print("All JSON files created successfully!")
print("="*60)


Processing plants...
  Created: IBIF_biodiversity_CO2_plants.json (2106 exchanges)
  Created: IBIF_biodiversity_NH3_plants.json (228 exchanges)
  Created: IBIF_biodiversity_NOx_plants.json (456 exchanges)
  Created: IBIF_biodiversity_LU_plants.json (9572 exchanges)
  Created: IBIF_biodiversity_all pressures_plants.json (12362 exchanges)

Processing vertebrates...
  Created: IBIF_biodiversity_CO2_vertebrates.json (2106 exchanges)
  Skipped: IBIF_biodiversity_NH3_vertebrates.json (0 exchanges)
  Skipped: IBIF_biodiversity_NOx_vertebrates.json (0 exchanges)
  Created: IBIF_biodiversity_LU_vertebrates.json (9572 exchanges)
  Created: IBIF_biodiversity_all pressures_vertebrates.json (11678 exchanges)

Processing overall...
  Created: IBIF_biodiversity_CO2_overall.json (2106 exchanges)
  Created: IBIF_biodiversity_NH3_overall.json (228 exchanges)
  Created: IBIF_biodiversity_NOx_overall.json (456 exchanges)
  Created: IBIF_biodiversity_LU_overall.json (9572 exchanges)
  Created: IBIF_biodiv